# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
# First, put in your user name below
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!

MYTABLE='paulboal' + '_data'



# Then, this code will automatically setup your 
import os
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2020.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2020'
password = 'SLUcor2020'
database = 'hds5210'

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + ':' + 
                     str(port) + '/' + 
                     database)

In [2]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [3]:
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'

### BEGIN SOLUTION
df = pd.read_csv(url)
df.to_sql(MYTABLE, conn, if_exists='replace')
### END SOLUTION

In [6]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,20))
assert(list(dxyz.columns) == ['index', 'Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

In [7]:
df.head()

,Provider_Name,NPI,CCN,Medicaid_EP_Hospital_Type,Street_Address,City,County,State,Zip_Code,Payment_Year_Number,Program_Type,Total_payments,Last_Program_Year,Last_Payment_Year,Last_Payment_Criteria,Most_Recent_Disbursement_Amount,LONGITUDE,LATITUDE,FID
0,Sutter Bay Hospitals,1659439834,50008,Acute Care Hospitals,CASTRO & DUBOCE STS,SAN FRANCISCO,San Francisco,CA,94114,3,Medicare/Medicaid,638474,2015,2017,MU,70942,-122.435802,37.769062,1
1,PRIME HEALTHCARE SERVICES - GARDEN GROVE LLC,1659538858,50230,Acute Care Hospitals,12601 GARDEN GROVE BLVD,GARDEN GROVE,Orange,CA,92843,4,Medicare/Medicaid,3947489,2014,2015,MU,394749,-117.913856,33.774499,2
2,ST MARY MEDICAL CENTER,1669456299,50300,Acute Care Hospitals,18300 US HIGHWAY 18,APPLE VALLEY,San Bernardino,CA,92307,3,Medicare/Medicaid,3062645,2014,2015,MU,340294,-117.262672,34.539918,3
3,MADERA COMMUNITY HOSPITAL,1669673646,50568,Acute Care Hospitals,1250 E ALMOND AVE,MADERA,Madera,CA,93637,4,Medicare/Medicaid,2057365,2015,2016,MU,205737,-120.045618,36.945447,4
4,Temecula Valley Hospital Inc,1679816201,50775,Acute Care Hospitals,31700 TEMECULA PKWY,TEMECULA,Riverside,CA,92592,1,Medicare/Medicaid,474790,2016,2017,AIU,474790,-117.117197,33.470664,5


## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [8]:
### BEGIN SOLUTION
maxpop = pd.read_sql_query("""
  SELECT m.*, p.*
  FROM {} m JOIN population p ON m.Zip_Code = p.zipcode
  ORDER BY p.population DESC
  """.format(MYTABLE), conn)

answer = ['1194016923']
### END SOLUTION

In [9]:
maxpop

,index,Provider_Name,NPI,CCN,Medicaid_EP_Hospital_Type,Street_Address,City,County,State,Zip_Code,...,LONGITUDE,LATITUDE,FID,zipcode,population,median_age,total_males,total_females,total_households,avg_household
0,277,"CPH HOSPITAL MANAGEMENT, LLC",1194016923,50771,Acute Care Hospitals,13100 STUDEBAKER RD,NORWALK,Los Angeles,CA,90650,...,-118.099176,33.912508,278,90650,105549,32.5,52364,53185,27130,3.83
1,259,COUNTY OF LOS ANGELES AUDITOR CONTROLLER,1134143415,50040,Acute Care Hospitals,14445 OLIVE VIEW DR,SYLMAR,Los Angeles,CA,91342,...,-118.448013,34.324172,260,91342,91725,31.9,45786,45939,23543,3.83
2,94,"VERITAS HEALTH SERVICES, INC.",1962407460,50586,Acute Care Hospitals,5451 WALNUT AVE,CHINO,San Bernardino,CA,91710,...,-117.686992,34.026355,95,91710,80358,33.0,42283,38075,21952,3.43
3,250,Jupiter Bellflower Doctors Hospital,1114021250,50531,Acute Care Hospitals,9542 ARTESIA BLVD,BELLFLOWER,Los Angeles,CA,90706,...,-118.130423,33.874755,251,90706,76615,31.9,37203,39412,23650,3.21
4,169,KAISER FOUNDATION HOSPITALS,1477608271,50723,Acute Care Hospitals,1011 BALDWIN PARK BLVD,BALDWIN PARK,Los Angeles,CA,91706,...,-117.985801,34.063442,170,91706,76571,30.5,37969,38602,17504,4.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,231,GARDENS REGIONAL HOSPITAL AND MEDICAL CENTER INC,1043373053,50575,Acute Care Hospitals,21530 PIONEER BLVD,HAWAIIAN GARDENS,Los Angeles,CA,90716,...,-118.081805,33.833514,232,90716,14184,28.3,7078,7106,3515,4.03
83,134,"PASADENA HOSPITAL ASSOCIATION, LTD.",1407828429,50438,Acute Care Hospitals,100 W CALIFORNIA BLVD,PASADENA,Los Angeles,CA,91105,...,-118.152322,34.135758,135,91105,11254,49.0,5432,5822,5213,2.07
84,110,AVALON MEDICAL DEVELOPMENT CORPORATION,1346250347,51307,Acute Care Hospitals,100 FALLS CANYON ROAD,AVALON,Los Angeles,CA,90704,...,-118.335504,33.339766,111,90704,4090,37.2,2101,1989,1629,2.50
85,38,UNIVERSITY OF SOUTHERN CALIFORNIA,1770728438,50660,Acute Care Hospitals,1441 EASTLAKE AVENUE,LOS ANGELES,Los Angeles,CA,90089,...,-118.204510,34.061751,39,90089,3217,19.3,1436,1781,31,1.94


In [10]:
assert(type(answer) == list)

### BEGIN HIDDEN TESTS
assert(answer == ['1194016923'])
### END HIDDEN TESTS

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [11]:
### BEGIN SOLUTION
answer = pd.read_sql_query("""
  SELECT z.Medicaid_EP_Hospital_Type, AVG(z.Payments / p.population) as Avg_Pay_per_Capita
  FROM (
      SELECT m.Medicaid_EP_Hospital_Type, m.Zip_Code, sum(Total_payments) as Payments
      FROM {} m
      GROUP BY m.Medicaid_EP_Hospital_Type, m.Zip_Code
  ) z JOIN population p ON z.Zip_Code = p.zipcode
  GROUP BY
    z.Medicaid_EP_Hospital_type
  """.format(MYTABLE), conn)
### END SOLUTIONS


In [12]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 17974.363)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 111.559)


In [13]:
answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,17974.362796
1,Children's Hospitals,111.559033


## Now using Pandas instead of SQL

In [20]:
### BEGIN SOLUTION
total_pay_per_zip = df.groupby(['Medicaid_EP_Hospital_Type','Zip_Code'])['Total_payments'].sum().reset_index()
total_pay_per_zip['Zip_Code'] = total_pay_per_zip['Zip_Code'].astype(str)

populations = pd.read_sql_query("SELECT * FROM population", conn)
populations['zipcode'] = populations['zipcode'].astype(str)

merged = total_pay_per_zip.merge(populations, how='left', left_on='Zip_Code', right_on='zipcode')

merged['Avg_Pay_per_Capita'] = merged['Total_payments'] / merged['population']

answer = merged.groupby('Medicaid_EP_Hospital_Type')['Avg_Pay_per_Capita'].mean().reset_index()

answer
### END SOLUTIONS


,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,17974.362790
1,Children's Hospitals,111.559021
